<a href="https://colab.research.google.com/github/AkramDataEngineering/Employee-Churn-Analysis-using-Rainforest-Model-using-PyCaret-Library/blob/main/Pilot_Analysis_EmployeeChurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to BIG Query

In [ ]:
#Libraries that we need
from google.cloud import bigquery
from google.colab import auth

#Authenticate
auth.authenticate_user()

#Initiate the Client for Big Query
project_id = "employee-churn-analysis-467505"
client = bigquery.Client(project=project_id,location="US")

In [ ]:
#Get the Dataset and Table
dataset_ref = client.dataset("employeedata", project=project_id)
table_ref = dataset_ref.table("tbl_hr_data")
table = client.get_table(table_ref)
table.schema

In [ ]:
#Get the Dataset and Table for Pilot employee

new_table_ref = dataset_ref.table("tbl_new_employees")
new_table = client.get_table(new_table_ref)
new_table.schema

In [ ]:
#CONVERT TO DATAFRAME
df_employees = client.list_rows(table).to_dataframe()
df_employees.head()

In [ ]:
#CONVERT TO DATAFRAME
df_pilot_employees = client.list_rows(new_table).to_dataframe()
df_pilot_employees.head()

# Build Model (Install Pycaret)


In [ ]:
# !pip install pycaret

# Code Model and Train

In [ ]:
df_employees.info()

In [ ]:
df_employees.columns

In [ ]:
#Get The Model(AUTO ML LIBRARY)
from pycaret.classification import *

In [ ]:
#Setup or Model
setup(df_employees, target= "Quit_the_Company" , session_id=123, ignore_features= ["employee_id"], categorical_features= ["salary","Departments"])

In [ ]:
compare_models()

In [ ]:
rf_model = create_model("rf")

In [ ]:
final_df = predict_model(rf_model)
final_df.head()

In [ ]:
final_df.info()

In [ ]:
# NOw the data has tated into Test Data ,Its Time to use the trained model on New data set of pilot employee

new_pred_df = predict_model(rf_model, data=df_pilot_employees)
new_pred_df.head()

In [ ]:
#Right back to Big Query
new_pred_df.to_gbq("employeedata.pilot_predictions",
                   project_id,
                   chunksize= None ,
                   if_exists= "replace")

In [ ]:
plot_model(rf_model, plot= "feature")

In [ ]:
#Create A Feature Table
rf_model.feature_names_in_

In [ ]:
rf_model.feature_importances_

In [ ]:
#Noew Combining this Feauture and Importance in  data frame
import pandas as pd
feature_table = pd.DataFrame(zip(rf_model.feature_names_in_, rf_model.feature_importances_ ), columns=["Feature","Importance"])
feature_table

In [ ]:
#Take this Table to Big Query
feature_table.to_gbq("employeedata.feature_table",
                   project_id,
                   chunksize= None ,
                   if_exists= "replace")